# Evaluation Part II
這個文本主要是介紹一些 LLM 回應訊息的模糊評估比較方法。例如 Rubric 評分方法、與專家回覆的比較方式。

### 環境設定
以下是環境的準備， 執行前請先在 `.env` 檔案內填入您自己的 openai api key。

In [1]:
import os
import openai
import sys
sys.path.append('../..')
import utils
import json
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

In [2]:
def get_completion_from_messages(messages, model="gpt-3.5-turbo-16k", temperature=0, max_tokens=13000):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, 
        max_tokens=max_tokens, 
    )
    return response.choices[0].message["content"]

### 從頭到尾來執行取得產品資訊以及最後的回覆使用者訊息

In [3]:
customer_msg = f"""
可以幫我介紹關於 smartX ProPhone 手機和 fotosnap 單眼相機的資訊嗎？"""

products_by_category = utils.get_products_from_query(customer_msg)
category_and_product_list = utils.read_string_to_list(products_by_category)
product_info = utils.get_mentioned_product_info(category_and_product_list)

# 取得助理回覆
assistant_answer = utils.answer_user_msg(user_msg=customer_msg,
                                                   product_info=product_info)

下方是回覆的內容

In [4]:
print(assistant_answer) 

當然可以！我們有兩款產品可以介紹給您。

首先是 SmartX ProPhone 手機。這款手機具有6.1吋的顯示器和128GB的儲存空間，讓您可以盡情儲存您的照片和影片。它還搭載了12MP的雙鏡頭，讓您拍攝出更清晰和生動的照片。此外，它還支援5G網路，讓您可以享受更快速的網路連接。SmartX ProPhone 手機的價格為899.99元，並且附帶一年的保固。

另一款產品是 Fotosnap 單眼相機。這款相機具有24.2MP的感光元件，可以捕捉出高品質的照片。它還支援1080p的影片拍攝，讓您可以錄製出清晰的視頻。相機配備了一個3吋的LCD螢幕，方便您即時檢視拍攝的照片和影片。此外，它還支援可更換鏡頭的功能，讓您可以根據不同的拍攝需求進行調整。Fotosnap 單眼相機的價格為599.99元，同樣附帶一年的保固。

請問您對這兩款產品有任何進一步的問題嗎？


### 根據提取的產品資訊，以 Rubic 評分標準評估 LLM 模型回答給使用這的回覆

In [5]:
cust_prod_info = {
    'customer_msg': customer_msg,
    'context': product_info
}

In [6]:
def eval_with_rubric(test_set, assistant_answer):

    cust_msg = test_set['customer_msg']
    context = test_set['context']
    completion = assistant_answer
    
    system_message = """\
    你是一個助理，透過檢視客服人員用來產生其回應的上下文， \
    評估客服人員回答使用者問題的表現。
    """

    user_message = f"""\
    你正在根據代理人用來回答問題的上下文，對提交的答案進行評估。
    以下是數據：
    [數據開始]
    ************
    [問題]: {cust_msg}
    ************
    [上下文]: {context}
    ************
    [提交的答案]: {completion}
    ************
    [數據結束]

    比較提交答案的事實內容與上下文，忽略風格、文法或標點符號的任何差異。
    回答以下問題（請記住你的答案，但是不要輸出）：
    - 助理的回答僅基於所提供的上下文嗎？（是或否）
    - 答案是否包含上下文中未提供的資訊？（是或否）
    - 回應與上下文之間是否有任何不符？（是或否）
    - 計算使用者問了多少個問題。（輸出一個數字）
    - 對於使用者提出的每個問題，是否有相對應的答案？
    - 在問的問題數中，有多少個問題得到了答案的回應？（只要輸出一個數字）

    請將你以上各問題的答案，以以下格式輸出：
    問題1：（是或否）
    問題2：（是或否）
    ...
    問題N：（是或否）
"""

    messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': user_message}
    ]

    response = get_completion_from_messages(messages)
    return response

執行評分並且列印結果

In [7]:
evaluation_output = eval_with_rubric(cust_prod_info, assistant_answer)
print(evaluation_output)

問題1：是
問題2：是
問題3：是
問題4：1
問題5：是
問題6：1


### 將 LLM 的回覆與專家答案做比較

In [8]:
# 這裏是專家的回覆
test_set_ideal = {
    'customer_msg': """ \
    可以幫我介紹關於 smartX ProPhone 手機和 fotosnap 單眼相機的資訊嗎？""",
    
    'ideal_answer':""" \
    當然可以！SmartX ProPhone 是一款功能強大的智慧型手機，具有先進的相機功能。 \
    例如，它擁有一個 1200 萬像素的雙鏡頭、5G 無線和 128GB 的儲存空間。 \
    它還有一個 6.1 吋的顯示器。價格是 $899.99。

    FotoSnap 數位單眼相機非常適合捕捉令人驚嘆的照片和影片。 \ 
    一些功能包括 1080p 的影片，3 吋液晶螢幕，2420 萬像素的感光元件，以及可更換鏡頭。 \
    價格是 $599.99。

    你對這些產品有什麼其他的問題嗎？
    或者有其他我可以幫你解答的問題嗎？
    """
}

### 檢查 LLM 的回應是否與專家的回覆相同或者不同意
這個驗證提示源於 [OpenAI evals](https://github.com/openai/evals/blob/main/evals/registry/modelgraded/fact.yaml) 專案並且加於翻譯以及中文化後發生的問題的一些微調.

[BLEU score](https://en.wikipedia.org/wiki/BLEU): 另一個可以驗證兩段文字的相似度的方式。

In [9]:
def eval_vs_ideal(test_set, assistant_answer):

    cust_msg = test_set['customer_msg']
    ideal = test_set['ideal_answer']
    completion = assistant_answer
    
    
    system_message = """\
    你是一個評分的助手，請透過實際回答與專家回答的內容比較來做評估。\
    評估結果你只能回答一個字母。
    """

    user_message= f"""\
    你正在針對原始問題來做實際回答以及專家回答間的差異評分，並且依照評分邏輯做為判斷邏輯。

    [評分資料開始]
    ************
    [原始問題]: {cust_msg}
    ************
    [專家回答]: {ideal}
    ************
    [實際回答]: {completion}
    ************
    [評分邏輯]:
    1. 將實際回答內容與專家的回答內容進行比較，忽略風格、語法、順序或標點的差異。
    2. 如果兩者回覆的內容，只是表達方式和詳細程度不同，請當作相同的回答。
    3. 實際回答可能是專家回答的子集或超集，或者可能與專家回答有衝突。
    ************
    [評分資料結束]
    
    請你確定下方哪種情況適用，並且只回覆 ABCDE 中的一個字元：
    (A) 實際回答是專家回答的子集。
    (B) 實際回答是專家回答的超集。
    (C) 實際回答和專家答案之間有不同意見。
    (D) 實際回答和專家答案相同
    (E) 答案有所不同，但從實際性的角度看，這些差異並不重要。

    """
    

    #print(f'user_msg: {user_message}')

    messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': user_message}
    ]

    response = get_completion_from_messages(messages)
    return response

### eval_vs_ideal 的測試

這是 LLM 的回覆

In [10]:
print(assistant_answer)

當然可以！我們有兩款產品可以介紹給您。

首先是 SmartX ProPhone 手機。這款手機具有6.1吋的顯示器和128GB的儲存空間，讓您可以盡情儲存您的照片和影片。它還搭載了12MP的雙鏡頭，讓您拍攝出更清晰和生動的照片。此外，它還支援5G網路，讓您可以享受更快速的網路連接。SmartX ProPhone 手機的價格為899.99元，並且附帶一年的保固。

另一款產品是 Fotosnap 單眼相機。這款相機具有24.2MP的感光元件，可以捕捉出高品質的照片。它還支援1080p的影片拍攝，讓您可以錄製出清晰的視頻。相機配備了一個3吋的LCD螢幕，方便您即時檢視拍攝的照片和影片。此外，它還支援可更換鏡頭的功能，讓您可以根據不同的拍攝需求進行調整。Fotosnap 單眼相機的價格為599.99元，同樣附帶一年的保固。

請問您對這兩款產品有任何進一步的問題嗎？


我們將專家回覆以及 LLM 回覆收入 eval_vs_ideal 進行評估

In [11]:
eval_vs_ideal(test_set_ideal, assistant_answer)

'B'

以下這是反例（完全不相關的訊息）的測試。

In [12]:
assistant_answer_2 = "life is like a box of chocolates"

In [13]:
eval_vs_ideal(test_set_ideal, assistant_answer_2)

'C'

# 其他範例
由於在將提示中文化的過程發生很多預期外的反應，所以以下是在重新撰寫提示時的簡易測試

In [14]:
ideal_answers = [
    """
    南洋地區（東南亞）擁有許多獨特的水果，以下是一些常見的例子：
    芒果：這個地區的芒果口感甜美，種類多樣。
    榴槤：被譽為"果王"，榴槤的口感獨特，味道濃郁。
    荔枝：具有甜美的味道和多汁的肉質，是夏季的理想水果。
    """,
    """
    芒果、荔枝、榴槤
    """
]

assist_msg = [
    """
    芒果、榴槤
    """, # 子集
    """
    南洋水果有： 芒果、榴槤、荔枝。
    """, # 完全相同
    """
    芒果、榴槤、荔枝、龍眼、香蕉、椰子、番荔枝、石榴、菠蘿、火龍果、木瓜
    """, # 超集
    """
    南洋有很多水果，例如：
    芒果: 甜美。
    榴槤： 果王
    荔枝：甜美多汁。
    """, # 含義相同
    """
    東南亞的代表性水果有： 蘋果、梨子、水蜜桃。
    """
]

test_set = {} 
test_set['customer_msg'] = """
南洋水果有哪些?
"""
test_set['ideal_answer'] = ideal_answers[0]
completion = assist_msg[0]

print(f'cust_msg: {test_set["customer_msg"]}')
print(f'ideal: {test_set["ideal_answer"]}')
print(f'completion: {completion}')

eval_vs_ideal(test_set, completion)

cust_msg: 
南洋水果有哪些?

ideal: 
    南洋地區（東南亞）擁有許多獨特的水果，以下是一些常見的例子：
    芒果：這個地區的芒果口感甜美，種類多樣。
    榴槤：被譽為"果王"，榴槤的口感獨特，味道濃郁。
    荔枝：具有甜美的味道和多汁的肉質，是夏季的理想水果。
    
completion: 
    芒果、榴槤
    


'A'